In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

%matplotlib inline

In [ ]:
parent_dir = os.path.split(os.getcwd())[0]

In [ ]:
zones = gpd.read_file(parent_dir + '\\Data\\New\\lms_zone_du_new.shp') # LMS Zone data

In [ ]:
car_driver = pd.read_csv(parent_dir + '\\Data\\New\\LMS_results\\lms-car-driver', index_col=0)

In [ ]:
car_pass = pd.read_csv(parent_dir + '\\Data\\LMS\\HB_matrices\\PAS_etmaal.dat', delimiter='\t')
train = pd.read_csv(parent_dir + '\\Data\\LMS\\HB_matrices\\TREIN_etmaal.dat', delimiter='\t')
bus = pd.read_csv(parent_dir + '\\Data\\LMS\\HB_matrices\\BUS_etmaal.dat', delimiter='\t')
hov = pd.read_csv(parent_dir + '\\Data\\LMS\\HB_matrices\\HOV_etmaal.dat', delimiter='\t')
bike = pd.read_csv(parent_dir + '\\Data\\LMS\\HB_matrices\\FIETS_etmaal.dat', delimiter='\t')
ebike = pd.read_csv(parent_dir + '\\Data\\LMS\\HB_matrices\\EBIKE_etmaal.dat', delimiter='\t')
walk = pd.read_csv(parent_dir + '\\Data\\LMS\\HB_matrices\\LOOP_etmaal.dat', delimiter='\t')

Merge ebike and normal bike

In [ ]:
bike_final = bike.merge(ebike, on=['Orig', 'Dest'], how='outer')

In [ ]:
bike_final['TRP'] = bike_final['TRP_x'] + bike_final['TRP_y']
bike_final = bike_final[['Orig', 'Dest', 'TRP']]

In [ ]:
mode_list = [car_driver, car_pass, train, bus, hov, bike_final, walk]

In [ ]:
labels = ['Car driver', 'Car passenger', 'Train', 'Bus',
          'Tram/Metro', '(e-)Bike', 'Walk']

Remove trips to/from other countries

In [ ]:
for i in range(len(mode_list)):
    mode_list[i] = mode_list[i][(mode_list[i].Orig.isin(np.arange(1, 1407))) & (mode_list[i].Dest.isin(np.arange(1, 1407)))]

In [ ]:
tb_data = zones[['ZONE_ID']]

## Modal split

Now determine the modal split, by looking at trips departing and arriving from each zone.

In [ ]:
orig_df = tb_data.copy()
dest_df = tb_data.copy()

for i in range(len(mode_list)):
    new_df = mode_list[i].groupby('Orig').sum()
    new_df = new_df.rename(columns={'TRP':labels[i] + '_o_a'})
    new_df = new_df.rename(columns={'tot':labels[i] + '_o_a'}).iloc[:, -1]


    orig_df = orig_df.merge(new_df, left_on='ZONE_ID', right_on='Orig', how='left')

    new_df = mode_list[i].groupby('Dest').sum()
    new_df = new_df.rename(columns={'TRP':labels[i] + '_d_a'})
    new_df = new_df.rename(columns={'tot':labels[i] + '_d_a'}).iloc[:, -1]


    dest_df = dest_df.merge(new_df, left_on='ZONE_ID', right_on='Dest', how='left')


Save absolute values to csv

In [ ]:
orig_df.to_csv(parent_dir + '\\Data\\New\\lms_modal_split_orig_abs.csv')
dest_df.to_csv(parent_dir + '\\Data\\New\\lms_modal_split_dest_abs.csv')


In [ ]:
tot_trips_o = orig_df.iloc[:, 1:].sum(axis=1)
tot_trips_d = dest_df.iloc[:, 1:].sum(axis=1)

In [ ]:
for i in range(len(mode_list)):

    orig_df.loc[:, labels[i] + '_o'] = np.round(orig_df[labels[i] + '_o_a'] / tot_trips_o * 100, 2)
    dest_df.loc[:, labels[i] + '_d'] = np.round(dest_df[labels[i] + '_d_a'] / tot_trips_d * 100, 2)


In [ ]:
orig_df

In [ ]:
dest_df

In [ ]:
modal_split_df = orig_df.iloc[:, np.r_[0, 8:15]]

In [ ]:
modal_split_df = modal_split_df.merge(dest_df.iloc[:, np.r_[0, 8:15]], on='ZONE_ID')

In [ ]:
modal_split_df

## To csv

In [ ]:
# modal_split_df.to_csv(parent_dir + '\\Data\\New\\lms_zone_lms_modal_split.csv')